In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
import torch
from tqdm.notebook import tqdm
import gc

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMultipleChoice
device='cuda'
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/deberta-v3-large-zeroshot-v2.0", token="YOUR_TOKEN")
model = AutoModelForMultipleChoice.from_pretrained("MoritzLaurer/deberta-v3-large-zeroshot-v2.0", token="YOUR_TOKEN", ignore_mismatched_sizes=True).to(device)

In [ ]:
swag = load_dataset("swag", "regular", trust_remote_code=True)

In [ ]:
def preprocess_function(examples, tokenizer):
    ending_names = ["ending0", "ending1", "ending2", "ending3"]
    first_sentences = [[context] * 4 for context in examples["sent1"]]
    question_headers = examples["sent2"]
    second_sentences = [
        [f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)
    ]
    
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # Tokenize and pad sequences to the maximum length in the batch
    tokenized_examples = tokenizer(first_sentences, second_sentences, padding="max_length", truncation=True)
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [ ]:
tokenized_swag = swag.map(lambda examples: preprocess_function(examples, tokenizer), batched=True)

In [ ]:
tokenized_swag["test"].format

In [ ]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
def predict(dataloader):
    all_predictions = []
    all_labels = []
    model.eval()  # Set the model to evaluation mode

    for batch in tqdm(dataloader, desc="Processing batches"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)  # Assuming labels are part of your batch

        with torch.no_grad():
            outputs = model(input_ids=input_ids.view(-1, 4, input_ids.size(-1)),
                            attention_mask=attention_mask.view(-1, 4, attention_mask.size(-1)))
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Move predictions and labels to CPU and append to lists
        all_predictions.extend(predictions.to('cpu').tolist())
        all_labels.extend(labels.to('cpu').tolist())

    return all_predictions, all_labels


In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    # This function will handle batching
    input_ids = torch.tensor([item['input_ids'] for item in batch])
    attention_mask = torch.tensor([item['attention_mask'] for item in batch])
    labels = torch.tensor([item['label'] for item in batch])
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
# Create a DataLoader for the test dataset
test_loader = DataLoader(tokenized_swag['validation'], batch_size=16, collate_fn=collate_fn)

# Assuming predict function is correctly defined
predictions, labels = predict(test_loader)  # Changed from test_batch to test_loader

In [ ]:
import evaluate

accuracy_metric = evaluate.load("accuracy")

def compute_accuracy(predictions, references):
    return accuracy_metric.compute(predictions=predictions, references=references)

test_accuracy = compute_accuracy(predictions, labels)
print(f"Test Accuracy: {test_accuracy['accuracy']}")

In [ ]:
print(predictions)

In [ ]:

# test_loader = DataLoader(tokenized_swag['test'], batch_size=16, collate_fn=collate_fn)

# # Assuming predict function is correctly defined
# predictions, labels = predict(test_loader)  # Changed from test_batch to test_loader
# df = pd.DataFrame(predictions, columns=['pred'])
# df.to_csv('predictions.csv', index=False)

In [ ]:
import pandas as pd

In [ ]:
# df = pd.DataFrame(predictions, columns=['pred'])
# df.to_csv('predictions.csv', index=False)